In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Data Processing

In [2]:
df = pd.read_csv("global_laptop_selling_data.csv")
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [3]:
def PrintInfo():
   print(pd.concat([df.isnull().sum(), df.dtypes, df.nunique()],axis=1, keys=['Null values', 'Datatype', 'Unique values']))

In [4]:
PrintInfo()

                  Null values Datatype  Unique values
laptop_ID                   0    int64           1303
Company                     0   object             19
Product                     0   object            618
TypeName                    0   object              6
Inches                      0  float64             18
ScreenResolution            0   object             40
Cpu                         0   object            118
Ram                         0   object              9
Memory                      0   object             39
Gpu                         0   object            106
OpSys                       0   object              9
Weight                      0   object            179
Price_euros                 0  float64            791


In [5]:
df.drop(['laptop_ID'], axis=1, inplace=True)
df.columns

Index(['Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu',
       'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros'],
      dtype='object')

## Ram

In [6]:
df['Ram'].unique()

array(['8GB', '16GB', '4GB', '2GB', '12GB', '6GB', '32GB', '24GB', '64GB'],
      dtype=object)

In [7]:
df['Ram'] = df['Ram'].str.replace('GB', '').astype(int)
df['Ram'].unique()

array([ 8, 16,  4,  2, 12,  6, 32, 24, 64])

## Weight

In [8]:
df['Weight'].head().unique()

array(['1.37kg', '1.34kg', '1.86kg', '1.83kg'], dtype=object)

In [9]:
df['Weight'] = df['Weight'].replace('kg', '', regex=True).astype(float)
df['Weight'].head().unique()

array([1.37, 1.34, 1.86, 1.83])

## ScreenResolution

In [10]:
df[['Res_Width', 'Res_Height']] = df['ScreenResolution'].str.extract(r'(\d+)x(\d+)').astype(int)
df[['Res_Width', 'Res_Height']].shape

(1303, 2)

In [11]:
df['Touchscreen'] = df['ScreenResolution'].str.contains('Touchscreen')
df['Touchscreen'].value_counts()

Touchscreen
False    1111
True      192
Name: count, dtype: int64

In [12]:
df['PanelType'] = df['ScreenResolution'].str.extract(r'(IPS Panel Retina Display|IPS Panel)')
df['PanelType'].fillna('Unknown', inplace=True)
df['PanelType'].value_counts(dropna=False)

PanelType
Unknown                     938
IPS Panel                   348
IPS Panel Retina Display     17
Name: count, dtype: int64

In [13]:
df.drop(['ScreenResolution'], axis=1, inplace=True)

## Memory

In [14]:
df['Memory'].value_counts()

Memory
256GB SSD                        412
1TB HDD                          223
500GB HDD                        132
512GB SSD                        118
128GB SSD +  1TB HDD              94
128GB SSD                         76
256GB SSD +  1TB HDD              73
32GB Flash Storage                38
2TB HDD                           16
64GB Flash Storage                15
512GB SSD +  1TB HDD              14
1TB SSD                           14
256GB SSD +  2TB HDD              10
1.0TB Hybrid                       9
256GB Flash Storage                8
16GB Flash Storage                 7
32GB SSD                           6
180GB SSD                          5
128GB Flash Storage                4
512GB SSD +  2TB HDD               3
16GB SSD                           3
512GB Flash Storage                2
1TB SSD +  1TB HDD                 2
256GB SSD +  500GB HDD             2
128GB SSD +  2TB HDD               2
256GB SSD +  256GB SSD             2
512GB SSD +  256GB SSD         

In [15]:
df['Memory'].replace({'Flash Storage':'SSD', '1\.0':'1','Hybrid':'HDD', ' \+  ':' + '},regex=True, inplace=True)
df['Memory'].replace({'1TB':'1000GB', '2TB':'2000GB'},regex=True, inplace=True)

In [16]:
df['Memory'].replace({
    '256GB SSD + 256GB SSD': '512GB SSD',
    '512GB SSD + 256GB SSD': '768GB SSD',
    '512GB SSD + 512GB SSD': '1000GB SSD',
    '1000GB HDD + 1000GB HDD': '2000GB HDD'},regex=True, inplace=True)

In [17]:
df['SSD(GB)'] = df['Memory'].str.extract(r'(\d+)GB SSD').fillna('0').astype(int)
df[df['SSD(GB)']!=0]['SSD(GB)'].value_counts().sum(), df['Memory'].str.contains('SSD').sum()

(918, 918)

In [18]:
df['HDD(GB)'] = df['Memory'].str.extract(r'(\d+)GB HDD').fillna('0').astype(int)
df[df['HDD(GB)']!=0]['HDD(GB)'].value_counts().sum(), df['Memory'].str.contains('HDD').sum()

(588, 588)

In [19]:
df.drop(['Memory'], axis=1, inplace=True)

## CPU

In [20]:
df['Cpu'].head(20).unique()

array(['Intel Core i5 2.3GHz', 'Intel Core i5 1.8GHz',
       'Intel Core i5 7200U 2.5GHz', 'Intel Core i7 2.7GHz',
       'Intel Core i5 3.1GHz', 'AMD A9-Series 9420 3GHz',
       'Intel Core i7 2.2GHz', 'Intel Core i7 8550U 1.8GHz',
       'Intel Core i5 8250U 1.6GHz', 'Intel Core i3 6006U 2GHz',
       'Intel Core i7 2.8GHz', 'Intel Core M m3 1.2GHz',
       'Intel Core i7 7500U 2.7GHz', 'Intel Core i7 2.9GHz',
       'Intel Core i3 7100U 2.4GHz'], dtype=object)

In [21]:
df[df['Cpu'].str.contains('(Intel|AMD)') == False]

,Company,Product,TypeName,Inches,Cpu,Ram,Gpu,OpSys,Weight,Price_euros,Res_Width,Res_Height,Touchscreen,PanelType,SSD(GB),HDD(GB)
1191,Samsung,Chromebook Plus,2 in 1 Convertible,12.3,Samsung Cortex A72&A53 2.0GHz,4,ARM Mali T860 MP4,Chrome OS,1.15,659.0,2400,1600,True,IPS Panel,32,0


In [22]:
df.drop(index=1191, inplace=True)

In [23]:
df['ClockSpeed(GHz)'] = df['Cpu'].str.extract(r'\s([\d\.]+)GHz').astype(float)
df['ClockSpeed(GHz)'].value_counts().sum(), df['Cpu'].str.contains('GHz').sum()

(1302, 1302)

In [24]:
def extract_cpu_series(cpu):
    if 'AMD' in cpu:
        return 'AMD'
    elif 'Intel Core i' in cpu:
        return 'Intel Core '+cpu.split(' ')[2]
    else:
        return 'Intel Other'
    
df['CPU'] = df['Cpu'].apply(extract_cpu_series)

In [25]:
df['CPU'].value_counts()

CPU
Intel Core i7    527
Intel Core i5    423
Intel Other      154
Intel Core i3    136
AMD               62
Name: count, dtype: int64

In [26]:
df.drop(['Cpu'], axis=1, inplace=True)

## GPU

In [27]:
list(df['Gpu'].head(40).unique())

['Intel Iris Plus Graphics 640',
 'Intel HD Graphics 6000',
 'Intel HD Graphics 620',
 'AMD Radeon Pro 455',
 'Intel Iris Plus Graphics 650',
 'AMD Radeon R5',
 'Intel Iris Pro Graphics',
 'Nvidia GeForce MX150',
 'Intel UHD Graphics 620',
 'Intel HD Graphics 520',
 'AMD Radeon Pro 555',
 'AMD Radeon R5 M430',
 'Intel HD Graphics 615',
 'AMD Radeon Pro 560',
 'Nvidia GeForce 940MX',
 'Intel HD Graphics 400',
 'Nvidia GeForce GTX 1050',
 'AMD Radeon R2',
 'AMD Radeon 530',
 'Nvidia GeForce 930MX',
 'Intel HD Graphics',
 'Intel HD Graphics 500']

In [28]:
df[df['Gpu'].str.contains('Nvidia')]['Gpu'].unique()

array(['Nvidia GeForce MX150', 'Nvidia GeForce 940MX',
       'Nvidia GeForce GTX 1050', 'Nvidia GeForce 930MX',
       'Nvidia GeForce GTX 1060', 'Nvidia GeForce 150MX',
       'Nvidia GeForce 920MX', 'Nvidia GeForce GTX 1070',
       'Nvidia GeForce GTX 1050 Ti', 'Nvidia GeForce MX130',
       'Nvidia GeForce GTX 940MX', 'Nvidia GeForce 920M',
       'Nvidia GeForce GTX 1050M', 'Nvidia GTX 980 SLI',
       'Nvidia GeForce GTX 1080', 'Nvidia Quadro M1200',
       'Nvidia GeForce GTX 950M', 'Nvidia GeForce GTX 980M',
       'Nvidia GeForce 930M', 'Nvidia GeForce GTX 940M',
       'Nvidia GeForce GT 940MX', 'Nvidia Quadro M2200M',
       'Nvidia Quadro M620', 'Nvidia GeForce GTX 965M',
       'Nvidia GeForce GTX1080', 'Nvidia GeForce GTX1050 Ti',
       'Nvidia GeForce GTX 960M', 'Nvidia Quadro M620M',
       'Nvidia GeForce GTX 970M', 'Nvidia GeForce GTX 960<U+039C>',
       'Nvidia GeForce GTX 960', 'Nvidia Quadro M520M',
       'Nvidia Quadro M2200', 'Nvidia Quadro M2000M',
       'N

In [29]:
def extract_gpu_series(gpu):
    if 'AMD' in gpu:
        return 'AMD'
    elif 'Intel Iris' in gpu:
        return 'Intel Iris'
    elif 'Intel' in gpu and 'Graphics' in gpu:
        return 'Intel HD Graphics'
    elif 'Nvidia' in gpu and 'Quadro' in gpu:
        return 'Nvidia Quadro'
    elif 'Nvidia' in gpu and 'GTX' in gpu:
        return 'Nvidia GTX'
    else:
        return 'Nvidia GeForce'
    
df['GPU'] = df['Gpu'].apply(extract_gpu_series)

In [30]:
df['GPU'].value_counts(dropna=False)

GPU
Intel HD Graphics    708
Nvidia GTX           239
AMD                  180
Nvidia GeForce       130
Nvidia Quadro         31
Intel Iris            14
Name: count, dtype: int64

In [31]:
df.drop(['Gpu', 'Product'], axis=1, inplace=True)

# After cleaning

In [32]:
price_col = df.pop('Price_euros')
df.insert(len(df.columns), 'Price', price_col)

In [33]:
print(f'Now Dataset contains {len(df.columns)} columns')

Now Dataset contains 16 columns


In [34]:
PrintInfo()

                 Null values Datatype  Unique values
Company                    0   object             19
TypeName                   0   object              6
Inches                     0  float64             18
Ram                        0    int64              9
OpSys                      0   object              9
Weight                     0  float64            171
Res_Width                  0    int64             13
Res_Height                 0    int64             10
Touchscreen                0     bool              2
PanelType                  0   object              3
SSD(GB)                    0    int64             11
HDD(GB)                    0    int64              7
ClockSpeed(GHz)            0  float64             25
CPU                        0   object              5
GPU                        0   object              6
Price                      0  float64            791


In [35]:
print(f'Dropping {df.duplicated().sum()} duplicate records')
df.drop_duplicates(inplace=True)

Dropping 29 duplicate records


# Saving DataFrame from next process

In [36]:
df.to_csv('Laptop price.csv', index=False)